# 🚀 Cryptocurrency Market Analysis: Phase 2-5
## Data Loading, Understanding, Preprocessing & Historical Analysis

---

### 📊 Project Overview

This analysis implements **Phase 2 through Phase 5** of our comprehensive cryptocurrency market analysis:

- **Phase 2**: Data Loading & Understanding
- **Phase 3**: Data Preprocessing & Feature Engineering  
- **Phase 4**: Exploratory Data Analysis (EDA)
- **Phase 5**: Historical Trend Analysis

### 🎯 Analysis Goals

1. **Load & Validate Data**: Import multiple cryptocurrency datasets
2. **Assess Data Quality**: Identify missing values and inconsistencies
3. **Engineer Features**: Create derived metrics for analysis
4. **Generate Statistics**: Comprehensive overview of market metrics
5. **Analyze Trends**: Historical price evolution and market patterns

---

## 📚 Import Required Libraries

Setting up our analysis environment with essential data science libraries.

In [ ]:
# Core data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
import glob

# Statistical libraries
from scipy import stats
import statsmodels.api as sm

# Plotly for interactive visualizations
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.6f}'.format)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

print("✅ All libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 Matplotlib version: {plt.matplotlib.__version__}")
print(f"📦 Seaborn version: {sns.__version__}")
print(f"📦 Analysis environment ready!")

## 📂 Initial Data Loading

Let's start by exploring available cryptocurrency datasets and setting up our data loading functions.

In [ ]:
# Define data directory path
DATA_DIR = 'd:\\Projects\\CoPilot\\Crypto\\bitcoin-crypto-market\\data\\raw'

# Check if data directory exists
if not os.path.exists(DATA_DIR):
    print(f"⚠️ Data directory not found: {DATA_DIR}")
    print("Creating directory structure...")
    os.makedirs(DATA_DIR, exist_ok=True)
    print("📁 Please place cryptocurrency CSV files in the data/raw directory")
else:
    print(f"✅ Data directory found: {DATA_DIR}")

# Function to discover available cryptocurrency files
def get_available_cryptos(data_dir):
    """Discover available cryptocurrency CSV files"""
    csv_files = glob.glob(os.path.join(data_dir, "*.csv"))
    crypto_names = []
    
    for file in csv_files:
        filename = os.path.basename(file)
        # Extract crypto name (assuming format: crypto_name.csv or coin_CryptoName.csv)
        if filename.startswith('coin_'):
            crypto_name = filename.replace('coin_', '').replace('.csv', '')
        else:
            crypto_name = filename.replace('.csv', '')
        crypto_names.append(crypto_name)
    
    return sorted(crypto_names), csv_files

# Function to load single cryptocurrency data
def load_crypto_data(filepath, crypto_name):
    """Load and validate single cryptocurrency dataset"""
    try:
        df = pd.read_csv(filepath)
        
        # Basic validation
        if df.empty:
            print(f"⚠️ {crypto_name}: Empty dataset")
            return None
        
        # Add crypto identifier
        df['Crypto'] = crypto_name
        
        # Convert Date column if exists
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.set_index('Date')
        
        print(f"✅ {crypto_name}: {len(df)} records loaded ({df.index.min().date()} to {df.index.max().date()})")
        return df
        
    except Exception as e:
        print(f"❌ Error loading {crypto_name}: {str(e)}")
        return None

# Discover available cryptocurrencies
available_cryptos, csv_files = get_available_cryptos(DATA_DIR)

print(f"\n🔍 Data Discovery Results:")
print(f"   • Available CSV files: {len(csv_files)}")
print(f"   • Cryptocurrency datasets: {len(available_cryptos)}")

if available_cryptos:
    print(f"\n📋 Available Cryptocurrencies:")
    for i, crypto in enumerate(available_cryptos, 1):
        print(f"   {i:2d}. {crypto}")
else:
    print("\n⚠️ No cryptocurrency CSV files found!")
    print("   Please ensure CSV files are placed in the data/raw directory")

## 🔄 Load Multiple Cryptocurrencies

Now let's load the key cryptocurrencies we'll focus on for our analysis.

In [ ]:
# Define key cryptocurrencies for analysis
# Prioritize major cryptocurrencies by market cap and data availability
KEY_CRYPTOS = [
    'Bitcoin', 'Ethereum', 'BinanceCoin', 'XRP', 'Cardano',
    'Solana', 'Dogecoin', 'Polkadot', 'Litecoin', 'ChainLink'
]

# Alternative names that might be in the dataset
CRYPTO_ALIASES = {
    'Bitcoin': ['bitcoin', 'BTC', 'btc'],
    'Ethereum': ['ethereum', 'ETH', 'eth'],
    'BinanceCoin': ['binancecoin', 'BNB', 'bnb', 'Binance'],
    'XRP': ['xrp', 'ripple', 'Ripple'],
    'Cardano': ['cardano', 'ADA', 'ada'],
    'Solana': ['solana', 'SOL', 'sol'],
    'Dogecoin': ['dogecoin', 'DOGE', 'doge'],
    'Polkadot': ['polkadot', 'DOT', 'dot'],
    'Litecoin': ['litecoin', 'LTC', 'ltc'],
    'ChainLink': ['chainlink', 'LINK', 'link']
}

def find_crypto_file(crypto_name, available_cryptos, csv_files):
    """Find the correct file for a cryptocurrency considering aliases"""
    # Direct match
    if crypto_name in available_cryptos:
        idx = available_cryptos.index(crypto_name)
        return csv_files[idx]
    
    # Check aliases
    if crypto_name in CRYPTO_ALIASES:
        for alias in CRYPTO_ALIASES[crypto_name]:
            if alias in available_cryptos:
                idx = available_cryptos.index(alias)
                return csv_files[idx]
    
    return None

# Load cryptocurrency data
crypto_data = {}
loading_summary = []

print("📊 Loading Key Cryptocurrencies...")
print("=" * 50)

for crypto_name in KEY_CRYPTOS:
    filepath = find_crypto_file(crypto_name, available_cryptos, csv_files)
    
    if filepath:
        df = load_crypto_data(filepath, crypto_name)
        if df is not None:
            crypto_data[crypto_name] = df
            loading_summary.append({
                'Cryptocurrency': crypto_name,
                'Records': len(df),
                'Start_Date': df.index.min(),
                'End_Date': df.index.max(),
                'Status': 'Loaded'
            })
        else:
            loading_summary.append({
                'Cryptocurrency': crypto_name,
                'Records': 0,
                'Start_Date': None,
                'End_Date': None,
                'Status': 'Failed'
            })
    else:
        print(f"⚠️ {crypto_name}: File not found")
        loading_summary.append({
            'Cryptocurrency': crypto_name,
            'Records': 0,
            'Start_Date': None,
            'End_Date': None,
            'Status': 'Not Found'
        })

# Create loading summary DataFrame
loading_df = pd.DataFrame(loading_summary)

print(f"\n📈 Loading Summary:")
print(f"   • Successfully loaded: {len(crypto_data)} cryptocurrencies")
print(f"   • Failed to load: {len(KEY_CRYPTOS) - len(crypto_data)} cryptocurrencies")

# Display detailed summary
display(loading_df)

## 🔍 Data Quality Assessment

Let's thoroughly examine the quality of our loaded cryptocurrency datasets.

In [ ]:
# Comprehensive data quality assessment
def assess_data_quality(crypto_data):
    """Perform comprehensive data quality assessment"""
    quality_report = []
    
    print("🔍 Data Quality Assessment")
    print("=" * 60)
    
    for name, df in crypto_data.items():
        print(f"\n📊 {name} Data Quality:")
        
        # Basic info
        total_records = len(df)
        date_range = (df.index.max() - df.index.min()).days
        
        # Missing values analysis
        missing_values = df.isnull().sum()
        total_missing = missing_values.sum()
        missing_percentage = (total_missing / (len(df) * len(df.columns))) * 100
        
        # Data type analysis
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Price consistency checks
        price_issues = 0
        if all(col in df.columns for col in ['High', 'Low', 'Open', 'Close']):
            # Check if High >= Low
            high_low_issues = (df['High'] < df['Low']).sum()
            # Check if High >= Open and High >= Close
            high_open_issues = (df['High'] < df['Open']).sum()
            high_close_issues = (df['High'] < df['Close']).sum()
            # Check if Low <= Open and Low <= Close
            low_open_issues = (df['Low'] > df['Open']).sum()
            low_close_issues = (df['Low'] > df['Close']).sum()
            
            price_issues = high_low_issues + high_open_issues + high_close_issues + low_open_issues + low_close_issues
        
        # Duplicate dates
        duplicate_dates = df.index.duplicated().sum()
        
        # Zero or negative values in price columns
        negative_values = 0
        zero_values = 0
        price_cols = ['Open', 'High', 'Low', 'Close']
        for col in price_cols:
            if col in df.columns:
                negative_values += (df[col] < 0).sum()
                zero_values += (df[col] == 0).sum()
        
        # Calculate data completeness
        completeness = ((len(df) * len(df.columns) - total_missing) / (len(df) * len(df.columns))) * 100
        
        # Store quality metrics
        quality_metrics = {
            'Cryptocurrency': name,
            'Total_Records': total_records,
            'Date_Range_Days': date_range,
            'Missing_Values': total_missing,
            'Missing_Percentage': missing_percentage,
            'Price_Logic_Issues': price_issues,
            'Duplicate_Dates': duplicate_dates,
            'Negative_Values': negative_values,
            'Zero_Values': zero_values,
            'Data_Completeness': completeness,
            'Numeric_Columns': len(numeric_cols)
        }
        quality_report.append(quality_metrics)
        
        # Print summary
        print(f"   • Records: {total_records:,}")
        print(f"   • Date range: {date_range} days")
        print(f"   • Missing values: {total_missing} ({missing_percentage:.2f}%)")
        print(f"   • Data completeness: {completeness:.2f}%")
        print(f"   • Price logic issues: {price_issues}")
        print(f"   • Duplicate dates: {duplicate_dates}")
        print(f"   • Negative/Zero values: {negative_values}/{zero_values}")
        
        # Show missing values by column if any
        if total_missing > 0:
            print(f"   • Missing by column:")
            for col, count in missing_values[missing_values > 0].items():
                pct = (count / len(df)) * 100
                print(f"     - {col}: {count} ({pct:.1f}%)")
    
    return pd.DataFrame(quality_report)

# Perform quality assessment
quality_df = assess_data_quality(crypto_data)

print(f"\n📋 Quality Assessment Summary:")
display(quality_df.round(2))

## 📊 Data Structure Exploration

Let's examine the detailed structure of our cryptocurrency datasets, using Bitcoin as our reference.

In [ ]:
# Detailed data structure exploration
def explore_data_structure(crypto_data):
    """Explore detailed structure of cryptocurrency datasets"""
    
    # Use Bitcoin as primary reference (most complete dataset typically)
    reference_crypto = None
    for crypto in ['Bitcoin', 'bitcoin', 'BTC', 'btc']:
        if crypto in crypto_data:
            reference_crypto = crypto
            break
    
    if not reference_crypto:
        reference_crypto = list(crypto_data.keys())[0]
    
    ref_df = crypto_data[reference_crypto]
    
    print(f"🔍 Detailed Structure Analysis - Reference: {reference_crypto}")
    print("=" * 60)
    
    # Basic structure
    print(f"\n📊 Dataset Dimensions:")
    print(f"   • Shape: {ref_df.shape}")
    print(f"   • Rows: {ref_df.shape[0]:,}")
    print(f"   • Columns: {ref_df.shape[1]}")
    
    # Column information
    print(f"\n📋 Column Information:")
    print(f"   • Column Names: {list(ref_df.columns)}")
    print(f"   • Data Types:")
    for col, dtype in ref_df.dtypes.items():
        print(f"     - {col}: {dtype}")
    
    # Date range information
    print(f"\n📅 Date Range:")
    print(f"   • Start Date: {ref_df.index.min().strftime('%Y-%m-%d')}")
    print(f"   • End Date: {ref_df.index.max().strftime('%Y-%m-%d')}")
    print(f"   • Total Days: {(ref_df.index.max() - ref_df.index.min()).days:,}")
    print(f"   • Index Type: {type(ref_df.index)}")
    
    # Sample data
    print(f"\n📈 Sample Data - First 5 Rows:")
    display(ref_df.head())
    
    print(f"\n📉 Sample Data - Last 5 Rows:")
    display(ref_df.tail())
    
    # Basic statistics for numeric columns
    print(f"\n📊 Basic Statistics:")
    numeric_cols = ref_df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        display(ref_df[numeric_cols].describe())
    
    # Check for expected cryptocurrency columns
    expected_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap']
    missing_cols = [col for col in expected_cols if col not in ref_df.columns]
    present_cols = [col for col in expected_cols if col in ref_df.columns]
    
    print(f"\n🔗 Standard Cryptocurrency Columns:")
    print(f"   • Present: {present_cols}")
    if missing_cols:
        print(f"   • Missing: {missing_cols}")
    
    return ref_df

# Explore structure
reference_df = explore_data_structure(crypto_data)

## 🔍 Missing Values Analysis

Let's conduct a comprehensive analysis of missing values across all cryptocurrencies.

In [ ]:
# Comprehensive missing values analysis
def analyze_missing_values(crypto_data):
    """Detailed missing values analysis across all cryptocurrencies"""
    
    print("🔍 Comprehensive Missing Values Analysis")
    print("=" * 60)
    
    # Overall missing values summary
    missing_summary = []
    
    for name, df in crypto_data.items():
        total_cells = len(df) * len(df.columns)
        missing_cells = df.isnull().sum().sum()
        missing_percentage = (missing_cells / total_cells) * 100
        
        missing_summary.append({
            'Cryptocurrency': name,
            'Total_Cells': total_cells,
            'Missing_Cells': missing_cells,
            'Missing_Percentage': missing_percentage,
            'Complete_Rows': len(df.dropna()),
            'Incomplete_Rows': len(df) - len(df.dropna())
        })
    
    missing_df = pd.DataFrame(missing_summary)
    missing_df = missing_df.sort_values('Missing_Percentage', ascending=False)
    
    print(f"\n📊 Missing Values Summary by Cryptocurrency:")
    display(missing_df)
    
    # Detailed missing values by column
    print(f"\n📋 Missing Values by Column:")
    
    all_columns = set()
    for df in crypto_data.values():
        all_columns.update(df.columns)
    
    column_missing = []
    for col in sorted(all_columns):
        col_data = []
        for name, df in crypto_data.items():
            if col in df.columns:
                missing_count = df[col].isnull().sum()
                missing_pct = (missing_count / len(df)) * 100
                col_data.append({
                    'Cryptocurrency': name,
                    'Column': col,
                    'Missing_Count': missing_count,
                    'Missing_Percentage': missing_pct
                })
        
        if col_data:
            col_df = pd.DataFrame(col_data)
            if col_df['Missing_Count'].sum() > 0:  # Only show columns with missing values
                print(f"\n   🔸 {col}:")
                for _, row in col_df[col_df['Missing_Count'] > 0].iterrows():
                    print(f"     - {row['Cryptocurrency']}: {row['Missing_Count']} ({row['Missing_Percentage']:.1f}%)")
    
    # Missing value patterns
    print(f"\n🔍 Missing Value Patterns:")
    
    for name, df in crypto_data.items():
        if df.isnull().sum().sum() > 0:
            print(f"\n   📊 {name}:")
            
            # Check for consecutive missing values
            for col in df.columns:
                if df[col].isnull().sum() > 0:
                    # Find consecutive missing value sequences
                    is_missing = df[col].isnull()
                    missing_groups = is_missing.ne(is_missing.shift()).cumsum()
                    consecutive_missing = is_missing.groupby(missing_groups).sum()
                    max_consecutive = consecutive_missing.max()
                    
                    print(f"     - {col}: Max consecutive missing = {max_consecutive}")
    
    # Data quality recommendations
    print(f"\n💡 Data Quality Recommendations:")
    
    high_missing = missing_df[missing_df['Missing_Percentage'] > 10]
    if not high_missing.empty:
        print(f"   ⚠️ High missing data (>10%):")
        for _, row in high_missing.iterrows():
            print(f"     - {row['Cryptocurrency']}: {row['Missing_Percentage']:.1f}% missing")
        print(f"   💡 Consider excluding or applying advanced imputation")
    
    moderate_missing = missing_df[(missing_df['Missing_Percentage'] > 1) & (missing_df['Missing_Percentage'] <= 10)]
    if not moderate_missing.empty:
        print(f"   🟡 Moderate missing data (1-10%):")
        for _, row in moderate_missing.iterrows():
            print(f"     - {row['Cryptocurrency']}: {row['Missing_Percentage']:.1f}% missing")
        print(f"   💡 Apply forward fill or interpolation")
    
    clean_data = missing_df[missing_df['Missing_Percentage'] <= 1]
    if not clean_data.empty:
        print(f"   ✅ Clean data (≤1% missing):")
        for _, row in clean_data.iterrows():
            print(f"     - {row['Cryptocurrency']}: {row['Missing_Percentage']:.1f}% missing")
    
    return missing_df

# Perform missing values analysis
missing_analysis = analyze_missing_values(crypto_data)

## 🧹 Data Preprocessing & Feature Engineering

Now let's clean our data and engineer features for analysis.

In [ ]:
# Data preprocessing and feature engineering
def preprocess_crypto_data(df, crypto_name):
    """Preprocess individual cryptocurrency data"""
    
    df_processed = df.copy()
    
    # 1. Handle missing values
    # Forward fill for small gaps (≤ 3 days)
    df_processed = df_processed.fillna(method='ffill', limit=3)
    
    # Interpolate remaining missing values for numeric columns
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_processed[col] = df_processed[col].interpolate(method='linear')
    
    # 2. Remove duplicates (keep first occurrence)
    df_processed = df_processed[~df_processed.index.duplicated(keep='first')]
    
    # 3. Sort by date
    df_processed = df_processed.sort_index()
    
    # 4. Feature Engineering - Price Features
    if 'Close' in df_processed.columns:
        # Daily returns
        df_processed['Daily_Return'] = df_processed['Close'].pct_change()
        
        # Log returns (better for analysis)
        df_processed['Log_Return'] = np.log(df_processed['Close'] / df_processed['Close'].shift(1))
        
        # Price change (absolute)
        if 'Open' in df_processed.columns:
            df_processed['Price_Change'] = df_processed['Close'] - df_processed['Open']
            df_processed['Price_Change_Pct'] = (df_processed['Close'] - df_processed['Open']) / df_processed['Open']
    
    # Price range features
    if all(col in df_processed.columns for col in ['High', 'Low', 'Open']):
        df_processed['Price_Range'] = df_processed['High'] - df_processed['Low']
        df_processed['Range_Pct'] = (df_processed['High'] - df_processed['Low']) / df_processed['Open'] * 100
    
    # 5. Moving Averages
    if 'Close' in df_processed.columns:
        df_processed['MA7'] = df_processed['Close'].rolling(window=7, min_periods=1).mean()
        df_processed['MA30'] = df_processed['Close'].rolling(window=30, min_periods=1).mean()
        df_processed['MA90'] = df_processed['Close'].rolling(window=90, min_periods=1).mean()
        df_processed['MA200'] = df_processed['Close'].rolling(window=200, min_periods=1).mean()
    
    # 6. Volatility Features
    if 'Daily_Return' in df_processed.columns:
        df_processed['Volatility_7'] = df_processed['Daily_Return'].rolling(window=7, min_periods=1).std()
        df_processed['Volatility_30'] = df_processed['Daily_Return'].rolling(window=30, min_periods=1).std()
        df_processed['Volatility_90'] = df_processed['Daily_Return'].rolling(window=90, min_periods=1).std()
    
    # 7. Cumulative Returns
    if 'Daily_Return' in df_processed.columns:
        df_processed['Cumulative_Return'] = (1 + df_processed['Daily_Return']).cumprod()
    
    # 8. Volume Features
    if 'Volume' in df_processed.columns:
        df_processed['Volume_Change'] = df_processed['Volume'].pct_change()
        df_processed['Volume_MA30'] = df_processed['Volume'].rolling(window=30, min_periods=1).mean()
        df_processed['Volume_Ratio'] = df_processed['Volume'] / df_processed['Volume_MA30']
    
    # 9. Date-based Features
    df_processed['Year'] = df_processed.index.year
    df_processed['Month'] = df_processed.index.month
    df_processed['DayOfWeek'] = df_processed.index.dayofweek
    df_processed['Quarter'] = df_processed.index.quarter
    df_processed['DayOfYear'] = df_processed.index.dayofyear
    
    # 10. Technical Indicators
    if 'Close' in df_processed.columns:
        # RSI (Relative Strength Index) - simplified version
        delta = df_processed['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14, min_periods=1).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14, min_periods=1).mean()
        rs = gain / loss
        df_processed['RSI'] = 100 - (100 / (1 + rs))
    
    print(f"✅ {crypto_name}: Preprocessing complete - {len(df_processed)} records, {len(df_processed.columns)} features")
    
    return df_processed

# Process all cryptocurrency data
processed_crypto_data = {}
feature_summary = []

print("🧹 Data Preprocessing & Feature Engineering")
print("=" * 60)

for name, df in crypto_data.items():
    processed_df = preprocess_crypto_data(df, name)
    processed_crypto_data[name] = processed_df
    
    # Track feature engineering results
    original_features = len(crypto_data[name].columns)
    new_features = len(processed_df.columns)
    added_features = new_features - original_features
    
    feature_summary.append({
        'Cryptocurrency': name,
        'Original_Features': original_features,
        'New_Features': new_features,
        'Added_Features': added_features,
        'Records_After_Processing': len(processed_df)
    })

# Display feature engineering summary
feature_df = pd.DataFrame(feature_summary)
print(f"\n📊 Feature Engineering Summary:")
display(feature_df)

# Show example of new features for Bitcoin
if 'Bitcoin' in processed_crypto_data:
    btc_processed = processed_crypto_data['Bitcoin']
    print(f"\n🔍 Example: New Features for Bitcoin")
    
    # Show original vs new columns
    original_cols = crypto_data['Bitcoin'].columns.tolist()
    new_cols = [col for col in btc_processed.columns if col not in original_cols]
    
    print(f"   • Original columns: {len(original_cols)}")
    print(f"   • New columns added: {len(new_cols)}")
    print(f"   • New features: {new_cols}")
    
    # Show sample of key features
    sample_features = ['Close', 'Daily_Return', 'MA30', 'Volatility_30', 'RSI']
    available_features = [col for col in sample_features if col in btc_processed.columns]
    
    print(f"\n📈 Sample Feature Values (Last 5 Days):")
    display(btc_processed[available_features].tail())

## 📊 Summary Statistics & Overview

Let's generate comprehensive summary statistics for all cryptocurrencies.

In [ ]:
# Generate comprehensive summary statistics
def calculate_comprehensive_stats(crypto_data):
    """Calculate comprehensive statistics for all cryptocurrencies"""
    
    summary_stats = []
    
    print("📊 Calculating Comprehensive Statistics")
    print("=" * 60)
    
    for name, df in crypto_data.items():
        stats_dict = {'Cryptocurrency': name}
        
        # Basic info
        stats_dict['Total_Records'] = len(df)
        stats_dict['Date_Range_Days'] = (df.index.max() - df.index.min()).days
        stats_dict['Start_Date'] = df.index.min()
        stats_dict['End_Date'] = df.index.max()
        
        # Price statistics
        if 'Close' in df.columns:
            stats_dict['Current_Price'] = df['Close'].iloc[-1]
            stats_dict['Min_Price'] = df['Close'].min()
            stats_dict['Max_Price'] = df['Close'].max()
            stats_dict['Avg_Price'] = df['Close'].mean()
            stats_dict['Median_Price'] = df['Close'].median()
            stats_dict['Price_Std'] = df['Close'].std()
            
            # Calculate total return
            first_price = df['Close'].dropna().iloc[0]
            last_price = df['Close'].dropna().iloc[-1]
            stats_dict['Total_Return_Pct'] = ((last_price - first_price) / first_price) * 100
        
        # Return statistics
        if 'Daily_Return' in df.columns:
            returns = df['Daily_Return'].dropna()
            stats_dict['Avg_Daily_Return'] = returns.mean()
            stats_dict['Median_Daily_Return'] = returns.median()
            stats_dict['Daily_Volatility'] = returns.std()
            stats_dict['Annualized_Volatility'] = returns.std() * np.sqrt(252)
            stats_dict['Skewness'] = returns.skew()
            stats_dict['Kurtosis'] = returns.kurtosis()
            stats_dict['Min_Daily_Return'] = returns.min()
            stats_dict['Max_Daily_Return'] = returns.max()
        
        # Volume statistics
        if 'Volume' in df.columns:
            volume_data = df['Volume'].dropna()
            stats_dict['Avg_Volume'] = volume_data.mean()
            stats_dict['Median_Volume'] = volume_data.median()
            stats_dict['Max_Volume'] = volume_data.max()
            stats_dict['Volume_Std'] = volume_data.std()
        
        # Market Cap statistics
        if 'Market Cap' in df.columns:
            mcap_data = df['Market Cap'].dropna()
            if len(mcap_data) > 0:
                stats_dict['Current_Market_Cap'] = mcap_data.iloc[-1]
                stats_dict['Avg_Market_Cap'] = mcap_data.mean()
                stats_dict['Max_Market_Cap'] = mcap_data.max()
        
        # Risk metrics
        if 'Daily_Return' in df.columns and 'Close' in df.columns:
            # Calculate maximum drawdown
            cumulative = (1 + df['Daily_Return'].fillna(0)).cumprod()
            running_max = cumulative.expanding().max()
            drawdown = (cumulative - running_max) / running_max
            stats_dict['Max_Drawdown'] = drawdown.min()
            
            # Sharpe ratio (assuming 0% risk-free rate)
            if stats_dict['Daily_Volatility'] > 0:
                stats_dict['Sharpe_Ratio'] = stats_dict['Avg_Daily_Return'] / stats_dict['Daily_Volatility'] * np.sqrt(252)
        
        summary_stats.append(stats_dict)
        
        print(f"✅ {name}: Statistics calculated")
    
    return pd.DataFrame(summary_stats)

# Calculate comprehensive statistics
comprehensive_stats = calculate_comprehensive_stats(processed_crypto_data)

# Display key metrics
print(f"\n📈 Key Performance Metrics:")
key_metrics = ['Cryptocurrency', 'Current_Price', 'Total_Return_Pct', 'Annualized_Volatility', 'Sharpe_Ratio', 'Max_Drawdown']
available_key_metrics = [col for col in key_metrics if col in comprehensive_stats.columns]
display(comprehensive_stats[available_key_metrics].round(2))

# Display price statistics
print(f"\n💰 Price Statistics:")
price_metrics = ['Cryptocurrency', 'Min_Price', 'Max_Price', 'Avg_Price', 'Current_Price']
available_price_metrics = [col for col in price_metrics if col in comprehensive_stats.columns]
display(comprehensive_stats[available_price_metrics].round(2))

# Display return statistics
print(f"\n📊 Return & Risk Statistics:")
return_metrics = ['Cryptocurrency', 'Avg_Daily_Return', 'Daily_Volatility', 'Min_Daily_Return', 'Max_Daily_Return']
available_return_metrics = [col for col in return_metrics if col in comprehensive_stats.columns]
display(comprehensive_stats[available_return_metrics].round(4))

# Identify top performers
if 'Total_Return_Pct' in comprehensive_stats.columns:
    best_performer = comprehensive_stats.loc[comprehensive_stats['Total_Return_Pct'].idxmax()]
    worst_performer = comprehensive_stats.loc[comprehensive_stats['Total_Return_Pct'].idxmin()]
    
    print(f"\n🏆 Performance Highlights:")
    print(f"   • Best Performer: {best_performer['Cryptocurrency']} ({best_performer['Total_Return_Pct']:.1f}% total return)")
    print(f"   • Worst Performer: {worst_performer['Cryptocurrency']} ({worst_performer['Total_Return_Pct']:.1f}% total return)")

# Identify risk characteristics
if 'Annualized_Volatility' in comprehensive_stats.columns:
    most_volatile = comprehensive_stats.loc[comprehensive_stats['Annualized_Volatility'].idxmax()]
    least_volatile = comprehensive_stats.loc[comprehensive_stats['Annualized_Volatility'].idxmin()]
    
    print(f"\n📊 Risk Characteristics:")
    print(f"   • Most Volatile: {most_volatile['Cryptocurrency']} ({most_volatile['Annualized_Volatility']:.1%} annual volatility)")
    print(f"   • Least Volatile: {least_volatile['Cryptocurrency']} ({least_volatile['Annualized_Volatility']:.1%} annual volatility)")

## 📈 Historical Trend Analysis

Let's analyze the historical price evolution and market trends.

In [ ]:
# Historical trend analysis - Bitcoin focus
def analyze_bitcoin_trends(bitcoin_df):
    """Comprehensive Bitcoin historical analysis"""
    
    print("📈 Bitcoin Historical Trend Analysis")
    print("=" * 50)
    
    # Basic trend statistics
    start_price = bitcoin_df['Close'].iloc[0]
    end_price = bitcoin_df['Close'].iloc[-1]
    total_return = ((end_price - start_price) / start_price) * 100
    
    print(f"📊 Bitcoin Price Evolution:")
    print(f"   • Start Price: ${start_price:,.2f}")
    print(f"   • End Price: ${end_price:,.2f}")
    print(f"   • Total Return: {total_return:,.1f}%")
    print(f"   • CAGR: {((end_price/start_price)**(1/(len(bitcoin_df)/365.25))-1)*100:.1f}%")
    
    # Year-over-year analysis
    bitcoin_df['Year'] = bitcoin_df.index.year
    yearly_stats = []
    
    for year in sorted(bitcoin_df['Year'].unique()):
        year_data = bitcoin_df[bitcoin_df['Year'] == year]
        if len(year_data) > 1:
            year_start = year_data['Close'].iloc[0]
            year_end = year_data['Close'].iloc[-1]
            year_return = ((year_end - year_start) / year_start) * 100
            year_volatility = year_data['Daily_Return'].std() * np.sqrt(252) * 100
            year_high = year_data['High'].max()
            year_low = year_data['Low'].min()
            
            yearly_stats.append({
                'Year': year,
                'Start_Price': year_start,
                'End_Price': year_end,
                'Year_Return_Pct': year_return,
                'Annual_Volatility_Pct': year_volatility,
                'Year_High': year_high,
                'Year_Low': year_low,
                'Days_Traded': len(year_data)
            })
    
    yearly_df = pd.DataFrame(yearly_stats)
    print(f"\n📅 Year-by-Year Performance:")
    display(yearly_df.round(2))
    
    return yearly_df

# Analyze Bitcoin trends if available
bitcoin_yearly = None
if 'Bitcoin' in processed_crypto_data:
    bitcoin_yearly = analyze_bitcoin_trends(processed_crypto_data['Bitcoin'])

# Plot Bitcoin price history with moving averages
if 'Bitcoin' in processed_crypto_data:
    btc_df = processed_crypto_data['Bitcoin']
    
    plt.figure(figsize=(15, 10))
    
    # Main price plot
    plt.subplot(2, 1, 1)
    plt.plot(btc_df.index, btc_df['Close'], label='Bitcoin Price', linewidth=2, color='orange')
    
    # Add moving averages
    if 'MA30' in btc_df.columns:
        plt.plot(btc_df.index, btc_df['MA30'], label='30-day MA', alpha=0.7, color='blue')
    if 'MA90' in btc_df.columns:
        plt.plot(btc_df.index, btc_df['MA90'], label='90-day MA', alpha=0.7, color='green')
    if 'MA200' in btc_df.columns:
        plt.plot(btc_df.index, btc_df['MA200'], label='200-day MA', alpha=0.7, color='red')
    
    plt.title('Bitcoin Price History with Moving Averages', fontsize=16, fontweight='bold')
    plt.ylabel('Price (USD)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')  # Log scale for better visualization
    
    # Volume subplot
    plt.subplot(2, 1, 2)
    if 'Volume' in btc_df.columns:
        plt.plot(btc_df.index, btc_df['Volume'], color='purple', alpha=0.7)
        plt.title('Bitcoin Trading Volume', fontsize=14, fontweight='bold')
        plt.ylabel('Volume', fontsize=12)
        plt.xlabel('Date', fontsize=12)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Multi-cryptocurrency comparison
def plot_normalized_comparison(crypto_data):
    """Plot normalized price comparison of all cryptocurrencies"""
    
    plt.figure(figsize=(15, 8))
    
    # Find common date range
    start_dates = [df.index.min() for df in crypto_data.values()]
    end_dates = [df.index.max() for df in crypto_data.values()]
    common_start = max(start_dates)
    common_end = min(end_dates)
    
    print(f"📊 Multi-Cryptocurrency Comparison")
    print(f"   • Common date range: {common_start.date()} to {common_end.date()}")
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(crypto_data)))
    
    for i, (name, df) in enumerate(crypto_data.items()):
        if 'Close' in df.columns:
            # Filter to common date range
            filtered_df = df[(df.index >= common_start) & (df.index <= common_end)]
            
            if len(filtered_df) > 0:
                # Normalize to base 100 (starting value = 100)
                normalized_prices = (filtered_df['Close'] / filtered_df['Close'].iloc[0]) * 100
                
                plt.plot(filtered_df.index, normalized_prices, 
                        label=name, linewidth=2, color=colors[i], alpha=0.8)
    
    plt.title('Normalized Cryptocurrency Price Comparison (Base = 100)', 
              fontsize=16, fontweight='bold')
    plt.ylabel('Normalized Price', fontsize=12)
    plt.xlabel('Date', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.tight_layout()
    plt.show()
    
    # Calculate relative performance
    performance_summary = []
    
    for name, df in crypto_data.items():
        if 'Close' in df.columns:
            filtered_df = df[(df.index >= common_start) & (df.index <= common_end)]
            if len(filtered_df) > 1:
                start_price = filtered_df['Close'].iloc[0]
                end_price = filtered_df['Close'].iloc[-1]
                total_return = ((end_price - start_price) / start_price) * 100
                
                performance_summary.append({
                    'Cryptocurrency': name,
                    'Start_Price': start_price,
                    'End_Price': end_price,
                    'Total_Return_Pct': total_return
                })
    
    performance_df = pd.DataFrame(performance_summary)
    performance_df = performance_df.sort_values('Total_Return_Pct', ascending=False)
    
    print(f"\n🏆 Performance Ranking (Common Period):")
    display(performance_df.round(2))
    
    return performance_df

# Generate multi-crypto comparison
if len(processed_crypto_data) > 1:
    performance_ranking = plot_normalized_comparison(processed_crypto_data)

In [ ]:
# Market capitalization evolution
def analyze_market_cap_evolution(crypto_data):
    """Analyze market capitalization trends"""
    
    print("💰 Market Capitalization Evolution Analysis")
    print("=" * 50)
    
    # Check which cryptocurrencies have market cap data
    cryptos_with_mcap = []
    for name, df in crypto_data.items():
        if 'Market Cap' in df.columns and not df['Market Cap'].isnull().all():
            cryptos_with_mcap.append(name)
    
    print(f"📊 Cryptocurrencies with Market Cap data: {len(cryptos_with_mcap)}")
    
    if cryptos_with_mcap:
        plt.figure(figsize=(15, 8))
        
        colors = plt.cm.tab10(np.linspace(0, 1, len(cryptos_with_mcap)))
        
        for i, name in enumerate(cryptos_with_mcap):
            df = crypto_data[name]
            mcap_data = df['Market Cap'].dropna()
            
            if len(mcap_data) > 0:
                plt.plot(mcap_data.index, mcap_data.values, 
                        label=name, linewidth=2, color=colors[i], alpha=0.8)
        
        plt.title('Market Capitalization Evolution', fontsize=16, fontweight='bold')
        plt.ylabel('Market Cap (USD)', fontsize=12)
        plt.xlabel('Date', fontsize=12)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.yscale('log')
        plt.tight_layout()
        plt.show()
        
        # Current market cap ranking
        current_mcaps = []
        for name in cryptos_with_mcap:
            df = crypto_data[name]
            current_mcap = df['Market Cap'].dropna().iloc[-1] if not df['Market Cap'].dropna().empty else 0
            current_mcaps.append({
                'Cryptocurrency': name,
                'Current_Market_Cap': current_mcap
            })
        
        mcap_ranking = pd.DataFrame(current_mcaps)
        mcap_ranking = mcap_ranking.sort_values('Current_Market_Cap', ascending=False)
        mcap_ranking['Market_Cap_Billions'] = mcap_ranking['Current_Market_Cap'] / 1e9
        
        print(f"\n📊 Current Market Cap Ranking:")
        display(mcap_ranking[['Cryptocurrency', 'Market_Cap_Billions']].round(2))
        
        return mcap_ranking
    else:
        print("⚠️ No market cap data available for analysis")
        return None

# Analyze market cap evolution
mcap_analysis = analyze_market_cap_evolution(processed_crypto_data)

## 📋 Phase 2-5 Summary

Let's summarize what we've accomplished in these phases.

In [ ]:
# Comprehensive summary of Phases 2-5
print("🎯 PHASE 2-5 COMPLETION SUMMARY")
print("=" * 60)

print(f"\n✅ PHASE 2: Data Loading & Understanding")
print(f"   • Loaded {len(crypto_data)} cryptocurrencies successfully")
print(f"   • Discovered {len(available_cryptos)} available datasets")
print(f"   • Established data loading framework")

print(f"\n✅ PHASE 3: Data Preprocessing & Feature Engineering")
processed_count = len(processed_crypto_data)
total_features = sum(len(df.columns) for df in processed_crypto_data.values())
print(f"   • Processed {processed_count} cryptocurrency datasets")
print(f"   • Generated {total_features} total features across all cryptocurrencies")
print(f"   • Added derived metrics: returns, volatility, moving averages, technical indicators")

print(f"\n✅ PHASE 4: Exploratory Data Analysis")
print(f"   • Generated comprehensive statistics for all cryptocurrencies")
print(f"   • Identified data quality issues and patterns")
print(f"   • Calculated risk and performance metrics")

print(f"\n✅ PHASE 5: Historical Trend Analysis")
print(f"   • Analyzed Bitcoin's historical price evolution")
print(f"   • Created normalized multi-cryptocurrency comparisons")
if mcap_analysis is not None:
    print(f"   • Examined market capitalization trends")
print(f"   • Generated year-over-year performance analysis")

# Key insights summary
print(f"\n💡 KEY INSIGHTS DISCOVERED:")

if not comprehensive_stats.empty:
    # Performance insights
    if 'Total_Return_Pct' in comprehensive_stats.columns:
        best_crypto = comprehensive_stats.loc[comprehensive_stats['Total_Return_Pct'].idxmax()]
        worst_crypto = comprehensive_stats.loc[comprehensive_stats['Total_Return_Pct'].idxmin()]
        print(f"   • Best performer: {best_crypto['Cryptocurrency']} ({best_crypto['Total_Return_Pct']:.1f}% return)")
        print(f"   • Worst performer: {worst_crypto['Cryptocurrency']} ({worst_crypto['Total_Return_Pct']:.1f}% return)")
    
    # Risk insights
    if 'Annualized_Volatility' in comprehensive_stats.columns:
        most_volatile = comprehensive_stats.loc[comprehensive_stats['Annualized_Volatility'].idxmax()]
        least_volatile = comprehensive_stats.loc[comprehensive_stats['Annualized_Volatility'].idxmin()]
        print(f"   • Most volatile: {most_volatile['Cryptocurrency']} ({most_volatile['Annualized_Volatility']:.1%})")
        print(f"   • Least volatile: {least_volatile['Cryptocurrency']} ({least_volatile['Annualized_Volatility']:.1%})")

# Data quality summary
total_missing = sum(df.isnull().sum().sum() for df in processed_crypto_data.values())
total_records = sum(len(df) for df in processed_crypto_data.values())
overall_completeness = ((total_records * len(processed_crypto_data[list(processed_crypto_data.keys())[0]].columns) - total_missing) / 
                       (total_records * len(processed_crypto_data[list(processed_crypto_data.keys())[0]].columns))) * 100

print(f"\n📊 DATA QUALITY METRICS:")
print(f"   • Overall data completeness: {overall_completeness:.1f}%")
print(f"   • Total records processed: {total_records:,}")
print(f"   • Average features per cryptocurrency: {total_features//processed_count}")

print(f"\n🚀 READY FOR NEXT PHASES:")
print(f"   • Phase 6: Bitcoin Dominance Analysis")
print(f"   • Phase 7: Volatility Analysis")
print(f"   • Phase 8: Correlation Analysis")
print(f"   • Phase 9: Seasonal Pattern Analysis")
print(f"   • Phase 10: Predictive Modeling")

print(f"\n" + "=" * 60)
print(f"📋 Phases 2-5 completed successfully! 🎉")